# Exercise 2

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("sommer_deger.csv")
df.head()

,instrument,treatment,outcome
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


- Instrument = 1 if the mother was offered a vitamin A shot for her baby
- Treatment = 1 if the baby recieved a vitamin A shot
- Outcome = 1 if baby died

#### 1. The first data scientist advised that one should compare the survival rate of babies whose mothers were offered Vitamin A shots to the survival rate of babies whose mothers were not offered a Vitamin A shot.

a. What percent of babies whose mothers were offered Vitamin A shots for their babies died? [3 pts]

In [48]:
print("Percentage of babies whose mothers were offered a Vitamin A shot died: ", np.mean(df[df['instrument'] == 1]["outcome"])) 

Percentage of babies whose mothers were offered a Vitamin A shot died:  0.003803538944931371


b. What percent of babies whose mothers were not offered Vitamin A shots for their babies died? [3 pts]


In [50]:
print("Percentage of babies whose mothers were not offered a Vitamin A shot died: ", np.mean(df[df['instrument'] == 0]["outcome"])) 

Percentage of babies whose mothers were not offered a Vitamin A shot died:  0.00638591646530894


c. What is the difference in mortality, and under what assumptions is the difference between these two percentages a valid estimate of the causal impact of receiving vitamin A shots on survival? [4 pts]

In [19]:
print("Difference in motality rate ", np.mean(df[df['instrument'] == 1]["outcome"]) - np.mean(df[df['instrument'] == 0]["outcome"])) 

Difference in motality rate  -0.002582377520377569


Under the assumptions that 1) mothers were randomly assigned to treatment and control groups and 2) the mothers who were offered the Vitamin A shots actually let the babies receive the shots can we estimate the causal impact of receiving Vitamin A shots on survival rate of babies using the difference between these two percentages.

#### 2. The second data scientist advised that one should compare the survival rates of babies who received Vitamin A shots to babies who did not receive Vitamin A shots.

a. What percent of babies who received Vitamin A shots died? [3 pts]


In [32]:
print("Percentage of babies that received a Vitamin A shot died: ", np.mean(df[df['treatment'] == 1]["outcome"])) 

Percentage of babies that received a Vitamin A shot died:  0.00124031007751938


b. What percent of babies who did not receive Vitamin A shots died? [3 pts]

In [31]:
print("Percentage of babies that did not receive a Vitamin A shot died: ", np.mean(df[df['treatment'] == 0]["outcome"])) 

Percentage of babies that did not receive a Vitamin A shot died:  0.007710430499036196


c. What is the difference in mortality, and under what assumptions is the difference between these two percentages a valid estimate of the causal impact of receiving vitamin A shots on survival? [4 pts]


In [28]:
print("Difference in motality rate ", np.mean(df[df['treatment'] == 1]["outcome"]) - np.mean(df[df['treatment'] == 0]["outcome"])) 

Difference in motality rate  -0.006470120421516816


Under the assumptions that 1) the babies are randomly assigned to treatment and control groups and 2) all babies who were in the treatment group received the treatment can we estimate the causal impact of receiving Vitamin A shots on survival rate of babies using the difference between these two percentages.

#### 3. The third data scientist advised that one should consider only babies whose mothers were offered Vitamin A shots, and compare babies who received shots to babies who did not receive shots.

a. What percent of babies who received Vitamin A shots died? [3 pts]

In [30]:
print("Percentage of babies who receievd Vitamin A died: ", np.mean(df[(df['treatment'] == 1) & (df['instrument'] == 1)]["outcome"])) 

Percentage of babies that received a Vitamin A shot and died:  0.00124031007751938


b. What percent of babies whose mothers were offered Vitamin A shots, but the mothers did not accept them, died? [3 pts]


In [36]:
print("Percentage of babies who did not receieve a Vitamin A died: ", np.mean(df[(df['treatment'] == 0) & (df['instrument'] == 1)]["outcome"])) 

Percentage of babies who did not receieve a Vitamin A died:  0.014055394791236048


c. What is the difference in mortality, and under what assumptions is the difference between these two percentages a valid estimate of the causal impact of receiving vitamin A shots on survival? [4 pts]

In [38]:
print("Difference in motality rate ", np.mean(df[(df['treatment'] == 1) & (df['instrument'] == 1)]["outcome"])
      - np.mean(df[(df['treatment'] == 0) & (df['instrument'] == 1)]["outcome"]))

Difference in motality rate  -0.012815084713716668


Under the assumptions that 1) the babies are randomly assigned to treatment and control groups and 2) self-selection bias is absent (mothers had no control over if babies would receive the Vitamin A shots) can we estimate the causal impact of receiving Vitamin A shots on survival rate of babies using the difference between these two percentages.

#### 4. The fourth data scientist suggested the following Wald estimator for the effect of Vitamin A shots on mortality
a. Compute the above Wald estimate for the given dataset. [2 pts]

In [55]:
p1 = np.mean(df[df['instrument'] == 1]["outcome"])
p2 =  np.mean(df[df['instrument'] == 0]["outcome"])
p3 =  len(df[(df['treatment'] == 1) & (df['instrument'] == 1)])/len(df[(df['instrument'] == 1)])
print("Wald estimate is ",(p1-p2)/p3)

Wald estimate is  -0.0032280386285732632


b. Under what assumptions is this estimate a valid estimate of the causal impact of vitamin A shots on survival? [4 pts]


Under the assumption that mortality rate is correlated with treatment but uncorrelated with instrumental variable: whether the mother was offered the Vitamin A shot is this estimate a valid estimate of the causal impact of Vitamin A shots on survival.

c. What is the standard error for the intent-to-treat estimate recommended by the first data scientist? What is the standard error for the Wald estimate recommended by the fourth data scientist? [5 pts]

i. Which one is larger and why? [4 pts]


In [11]:
import statsmodels.api as sm
X = df['instrument']
X= sm.add_constant(X)
y = df['outcome']
lm_1st = sm.OLS(y, X).fit()
lm_1st.bse

const         0.000660
instrument    0.000923
dtype: float64

The standard error of intent-to-treat estimate is 0.000923 for the approach recommended by the first data scientist.

In [12]:
y_1 =  df['treatment'] 
lm_1st_stage = sm.OLS(y_1, X).fit()
X_hat = lm_1st_stage.predict(X) 
X_hat= sm.add_constant(X_hat)
lm_2nd_stage = sm.OLS(y, X_hat).fit()
lm_2nd_stage.bse

const    0.000660
0        0.001154
dtype: float64

The standard error of Wald estimate is 0.001154 for the approach recommended by the fourth data scientist. This standard error is larger than previous one because the formula of standard error of IV is $\frac{\sigma^2}{SST\times R_{x,z}^2}$ whereas the one of OLS estimate is just $\frac{\sigma^2}{SST}$. As $R^2$ is between 0 and 1, the standard error of IV is inflated.

ii. Why might these standard errors be biased? What information would
you ideally want to have to address this bias? [5 pts]

These standard errors might be biased because the instrumental variable in this case is likely to be correlated with the response variable. Intent-to-treat and the survival rate seem to come hand in hand. To solve this issue, we need to add other factors to the regression to make sure that instrumental variable is independent of the error term of OLS. 

# Exercise 3

Read the paper “The Design Versus the Analysis of Observational Studies for Causal Effects:
Parallels With the Design of Randomized Trials” by Donald Rubin.
Write a short reflection paragraph up to half a page with your comments. You can use bullet
points to write your comments. Comments can be 1) points that resonated most with you, 2)
points of disagreement, 3) a comparison to what you’ve learned in previous classes, or 4)
anything else you want to comment on as long as they demonstrate that you read and thought
about the paper. [10 pts]

- The approach introduced in this research paper reminds me of A/B testing with matching technique that I have learned in the previous class. 
A/B testing with matching is a randomized experiment where we match similar subjects using propensity score and assign them to control and treatment groups randomly. The propensity score measures how similar the two subjects are in terms of certain observed characteristics of the subjects. The application in this paper also uses propensity score to create treatment and control subgroups to approximate randomized experiments. The paper emphasizes that matching and group assignment should be conducted without access to the outcome data to ensure objectivity of the result.

- What resonated most with me is author's strong opposition to manipulating the result by redesigning the experiment after acquiring the outcome data. 

- The design of what variables should be included in the propensity model takes careful thoughts. It requires deep industry knowledge and compliance with the law.